In [17]:
import os
import shutil
import random
from pathlib import Path
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict
import subprocess

In [7]:
!git clone https://github.com/ultralytics/yolov5 yolov5
!pip install -qr yolov5/requirements.txt

# Verify YOLOv5 installation
!python --version
!python -c "import torch; print('PyTorch:', torch.__version__)"
!ls yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.
Python 3.10.12
PyTorch: 2.5.0+cu121
benchmarks.py	 data	     LICENSE	     README.md	       train.py        yolov5s.pt
CITATION.cff	 detect.py   models	     README.zh-CN.md   tutorial.ipynb
classify	 export.py   __pycache__     requirements.txt  utils
CONTRIBUTING.md  hubconf.py  pyproject.toml  segment	       val.py


In [8]:
def create_small_dataset(original_base_dir: Path, small_base_dir: Path, samples_per_split: Dict[str, int]):
    """
    Create a smaller version of the dataset for testing.

    Args:
        original_base_dir: Path to original dataset
        small_base_dir: Path to create small dataset
        samples_per_split: Dict with number of samples for each split (train, val, test)
    """
    print("Creating small dataset...")

    # Create directory structure
    for split in ['train', 'val', 'test']:
        for subdir in ['images', 'labels']:
            os.makedirs(small_base_dir / split / subdir, exist_ok=True)

    # Copy subset of files for each split
    for split in ['train', 'val', 'test']:
        # Get list of all images
        image_dir = original_base_dir / split / 'images'
        all_images = list(image_dir.glob('*.jpg'))

        # Randomly select subset
        n_samples = min(samples_per_split[split], len(all_images))
        selected_images = random.sample(all_images, n_samples)

        print(f"Copying {n_samples} files for {split} split...")

        # Copy selected files
        for img_path in selected_images:
            # Copy image
            shutil.copy2(img_path, small_base_dir / split / 'images' / img_path.name)

            # Copy corresponding label
            label_path = original_base_dir / split / 'labels' / f"{img_path.stem}.txt"
            if label_path.exists():
                shutil.copy2(label_path, small_base_dir / split / 'labels' / f"{img_path.stem}.txt")

    # Create data.yaml for small dataset
    yaml_content = {
        'train': str(small_base_dir / 'train' / 'images'),
        'val': str(small_base_dir / 'val' / 'images'),
        'test': str(small_base_dir / 'test' / 'images'),
        'nc': 6,
        'names': ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
    }

    import yaml
    with open(small_base_dir / 'data.yaml', 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)

    print("Small dataset created successfully!")
    return small_base_dir / 'data.yaml'

In [9]:
def verify_dataset(base_dir: Path):
    """Verify dataset structure and print statistics."""
    print("\nVerifying dataset structure:")

    # Required directories
    required_dirs = ['train/images', 'train/labels',
                    'val/images', 'val/labels',
                    'test/images', 'test/labels']

    dataset_info = {}

    for dir_path in required_dirs:
        full_path = base_dir / dir_path
        if not full_path.exists():
            print(f"Missing directory: {full_path}")
            return None

        # Count files in directory
        n_files = len(list(full_path.glob('*')))
        print(f"Found {dir_path}: {n_files} files")
        dataset_info[dir_path] = n_files

    # Check for yaml file
    yaml_path = base_dir / 'trashnet.yaml'
    if yaml_path.exists():
        print(f"Found data.yaml at {yaml_path}")
    else:
        print(f"Missing data.yaml at {yaml_path}")
        return None

    return dataset_info

Load Pretrained YOLOv5 model

In [10]:
class GarbageClassifier:
    """Garbage classification using YOLOv5."""

    def __init__(self):
        """Initialize classifier with basic setup."""
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")

        # Load model
        print("Loading YOLOv5 model...")
        self.model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        self.model.to(self.device)
        print("Model loaded successfully")

        # Setup directories
        self.results_dir = os.path.abspath("training_results")
        os.makedirs(self.results_dir, exist_ok=True)

        # Class names
        self.class_names = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]

    def train(self, data_yaml: str, epochs: int, batch_size: int = 16) -> str:
        """Train model and return results path."""
        if not os.path.exists(data_yaml):
            raise FileNotFoundError(f"data.yaml not found at {data_yaml}")

        exp_name = f"exp_{epochs}_epochs"
        exp_path = os.path.join(self.results_dir, exp_name)

        # Modified training command without --verbose
        cmd = (f"cd yolov5 && "
               f"python train.py "
               f"--img 640 "
               f"--batch {batch_size} "
               f"--epochs {epochs} "
               f"--data {str(Path(data_yaml).absolute())} "
               f"--weights yolov5s.pt "
               f"--project {str(Path(self.results_dir).absolute())} "
               f"--name {exp_name} "
               f"--exist-ok "  # Add this to overwrite existing experiments
               f"--patience 0")  # Disable early stopping

        print(f"\nStarting training for {epochs} epochs...")
        print(f"Training command: {cmd}")

        try:
            process = subprocess.run(
                cmd,
                shell=True,
                capture_output=True,
                text=True,
                check=True
            )
            print("Training output:")
            print(process.stdout)
            if process.stderr:
                # Filter out wandb warnings
                stderr_lines = [line for line in process.stderr.split('\n')
                              if 'wandb' not in line.lower() and line.strip()]
                if stderr_lines:
                    print("Errors:", '\n'.join(stderr_lines))
        except subprocess.CalledProcessError as e:
            print(f"Training failed with error: {e}")
            if e.output:
                print("Output:", e.output)
            if e.stderr:
                print("Error output:", e.stderr)
            return None

        # Check for results file
        results_file = os.path.join(exp_path, 'results.csv')
        if not os.path.exists(results_file):
            print(f"Warning: No results file found at {results_file}")
            return None

        return exp_path

In [11]:
# def create_and_verify_yaml(base_dir: str) -> str:
#     """Create and verify data.yaml file."""
#     # Convert paths to absolute and handle spaces correctly
#     train_path = str(Path(base_dir) / 'train' / 'images')
#     val_path = str(Path(base_dir) / 'val' / 'images')
#     test_path = str(Path(base_dir) / 'test' / 'images')

#     # Verify directories exist
#     for path in [train_path, val_path, test_path]:
#         if not os.path.exists(path):
#             raise FileNotFoundError(f"Directory not found: {path}")

#     data_config = {
#         'train': train_path,
#         'val': val_path,
#         'test': test_path,
#         'nc': 6,
#         'names': ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
#     }

#     yaml_path = str(Path(base_dir) / 'trashnet.yaml')

#     # Save YAML file
#     import yaml
#     with open(yaml_path, 'w') as f:
#         yaml.dump(data_config, f, default_flow_style=False)

#     # Verify YAML file was created and is readable
#     with open(yaml_path, 'r') as f:
#         loaded_config = yaml.safe_load(f)
#         print("\nCreated data.yaml with contents:")
#         print(yaml.dump(loaded_config, default_flow_style=False))

#     return yaml_path

In [12]:
def train_and_plot(classifier: GarbageClassifier, data_yaml: str, epochs_list: List[int]):
    """Train models and collect results."""
    results = {}

    for epochs in epochs_list:
        exp_path = classifier.train(data_yaml, epochs)
        if exp_path is None:
            print(f"Skipping results for {epochs} epochs due to training failure")
            continue

        results_file = os.path.join(exp_path, 'results.csv')
        try:
            df = pd.read_csv(results_file)
            results[epochs] = {
                'train_loss': df['train/box_loss'].values,
                'val_loss': df['val/box_loss'].values,
                'mAP50': df['metrics/mAP_0.5'].values  # Keep all values for plotting
            }
            print(f"\nTraining completed for {epochs} epochs")
            print(f"Final mAP50: {results[epochs]['mAP50'][-1]:.4f}")
        except Exception as e:
            print(f"Error processing results for {epochs} epochs: {str(e)}")

    return results

In [13]:
def plot_results(results: Dict):
    """Plot training vs validation metrics."""
    if not results:
        print("No results to plot")
        return

    try:
        plt.figure(figsize=(10, 5))

        # Get the epochs with the most data points (usually the longest training run)
        best_epoch = max(results.keys(), key=lambda k: len(results[k]['train_loss']))
        epochs_range = range(1, len(results[best_epoch]['train_loss']) + 1)

        # Plot training and validation loss
        plt.plot(epochs_range, results[best_epoch]['train_loss'],
                label='Training Loss', color='blue')
        plt.plot(epochs_range, results[best_epoch]['val_loss'],
                label='Validation Loss', color='red')

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training and Validation Loss Over Time')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Error plotting results: {str(e)}")

In [14]:
def main():
    try:
        # Original dataset path
        original_base_dir = Path("/content/drive/MyDrive/ColabNotebooks_New/data/TrashNet")

        # Create small dataset
        small_base_dir = Path("/content/small_trashnet")
        samples_per_split = {
            'train': 100,  # 100 images for training
            'val': 50,     # 50 images for validation
            'test': 20     # 20 images for testing
        }

        # Create small dataset and get yaml path
        #data_yaml = create_small_dataset(original_base_dir, small_base_dir, samples_per_split)

        data_yaml = "/content/small_trashnet/data.yaml"

        # Initialize classifier
        classifier = GarbageClassifier()

        # Train with a single epoch
        epochs_list = [1]
        results = train_and_plot(classifier, str(data_yaml), epochs_list)

        # Plot results
        plot_results(results)

    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        import traceback
        traceback.print_exc()

In [15]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [18]:
if __name__ == "__main__":
    main()

Using device: cpu
Loading YOLOv5 model...


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-12 Python-3.10.12 torch-2.5.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Model loaded successfully

Starting training for 1 epochs...
Training command: cd yolov5 && python train.py --img 640 --batch 16 --epochs 1 --data /content/small_trashnet/data.yaml --weights yolov5s.pt --project /content/training_results --name exp_1_epochs --exist-ok --patience 0
Training output:

Errors: 2024-11-12 21:27:37.852495: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 21:27:37.888062: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 21:27:37.898510: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/small_tras